In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np

In [3]:
#Get ALL the html data from the page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.content)

In [44]:
#Get only the tables from the whole data
html_tables = soup.find_all('table')

#Filter on the table 3
table2 = html_tables[3]


In [42]:
#get the column names

dfcolumns = []
flight_no = []

#select all the rows (tr) of the table
for items in table2.find_all("tr"):
    #select all the headers (th) from the rows
    for rows in items.find_all("th"):
        if rows.sup:
            rows.sup.extract()
        rows = rows.text
        if not(rows.strip().isdigit()):
            dfcolumns.append(rows.strip())
        else:
            flight_no.append(rows.strip())


print("Columns : ", dfcolumns)


Columns :  ['Flight No.', 'Date andtime (UTC)', 'Version,Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


In [16]:
#get the content of the table


#Create empty lists containing the values, for each column
content = []
date_and_time = []
version_booster = []
launch_site = []
payload = []
payload_mass = []
orbit = []
customer = []
launchoutcome = []
boosterlanding = []


#Extract all the content in a list

#tr = row, with multiple elements. 
for items in table2.find_all("tr"):
    #td = only one element
    for rows in items.find_all("td"):
        if rows.sup:
            rows.sup.extract()
        if rows.br:
            rows.br.extract()
        rows = rows.text
        rows = rows.replace("\n", "")
        content.append(rows)
        
i = 0
for items in content:
        if i in np.arange(0, len(content), len(columns)):
            date_and_time.append(items)
        if i in np.arange(1, len(content), len(columns)):
            version_booster.append(items)    
        if i in np.arange(2, len(content), len(columns)):
            launch_site.append(items)    
        if i in np.arange(3, len(content), len(columns)):
            payload.append(items)
        if i in np.arange(4, len(content), len(columns)):
            payload_mass.append(items)
        if i in np.arange(5, len(content), len(columns)):
            orbit.append(items)
        if i in np.arange(6, len(content), len(columns)):
            customer.append(items)
        if i in np.arange(7, len(content), len(columns)):
            launchoutcome.append(items)
        if i in np.arange(8, len(content), len(columns)):
            boosterlanding.append(items)
        i +=1
            
print(date_and_time)
print(version_booster)
print(launchoutcome)
print(boosterlanding)
print(payload)


['6 January 2014,22:06', '18 April 2014,19:25', '14 July 2014,15:15', '5 August 2014,08:00', '7 September 2014,05:00', '21 September 2014,05:52']
['F9 v1.1', 'F9 v1.1', 'F9 v1.1', 'F9 v1.1', 'F9 v1.1B1011', 'F9 v1.1B1010']
['Success', 'Success', 'Success', 'Success', 'Success', 'Success']
['No attempt', 'Controlled(ocean) ', 'Controlled(ocean)', 'No attempt', 'No attempt', 'Uncontrolled(ocean)']
['Thaicom 6', 'SpaceX CRS-3(Dragon C105)', 'Orbcomm-OG2-1(6 satellites)', 'AsiaSat 8', 'AsiaSat 6', 'SpaceX CRS-4(Dragon C106.1)']


In [37]:
#Create dictionnary containing all the data

dict = {"Flight No" : flight_no, "Date and time" : date_and_time, "Booster version" : version_booster, 
        "Launch site" : launch_site, "Payload" : payload, "Payload mass" : payload_mass, "Orbit" : orbit, 
        "Customer" : customer, "Launch outcome" : launchoutcome, "Booster landing" : boosterlanding}


In [43]:
#Convert dictionnary into dataframe

df = pd.DataFrame(dict)
df = df.set_index("Flight No")
df

,Date and time,Booster version,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
Flight No,,,,,,,,,
8,"6 January 2014,22:06",F9 v1.1,"CCAFS,SLC-40",Thaicom 6,"3,325 kg (7,330 lb)",GTO,Thaicom,Success,No attempt
9,"18 April 2014,19:25",F9 v1.1,"Cape Canaveral,LC-40",SpaceX CRS-3(Dragon C105),"2,296 kg (5,062 lb)",LEO (ISS),NASA (CRS),Success,Controlled(ocean)
10,"14 July 2014,15:15",F9 v1.1,"Cape Canaveral,LC-40",Orbcomm-OG2-1(6 satellites),"1,316 kg (2,901 lb)",LEO,Orbcomm,Success,Controlled(ocean)
11,"5 August 2014,08:00",F9 v1.1,"Cape Canaveral,LC-40",AsiaSat 8,"4,535 kg (9,998 lb)",GTO,AsiaSat,Success,No attempt
12,"7 September 2014,05:00",F9 v1.1B1011,"Cape Canaveral,LC-40",AsiaSat 6,"4,428 kg (9,762 lb)",GTO,AsiaSat,Success,No attempt
13,"21 September 2014,05:52",F9 v1.1B1010,"Cape Canaveral,LC-40",SpaceX CRS-4(Dragon C106.1),"2,216 kg (4,885 lb)",LEO (ISS),NASA (CRS),Success,Uncontrolled(ocean)
